# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import genpareto, norm, beta, cauchy, t
from GBEX import GBEX
from GB_GPD import GB_GPD
import quantile_forest as qf
from tqdm import tqdm
from utils import *
import plotly.graph_objects as go
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error
import plotly.express as px

# Optimal number of trees : CV

## Model 1 : Poisson with covariates-dependent parameter

### Experiments on trees hyperparameters

In [3]:
X = (np.random.random((2000, 40))-0.5)*2
Y = np.zeros(2000)

lamb = lambda x: 1 + ((x[0]*x[1] < 0) | (x[2] > 0)).astype(float)
for i in range(2000):
    Y[i] = np.random.exponential(1/lamb(X[i]))

In [4]:
cv_results = {}

depth_pairs = [
    (1, 0),
    (1, 1),
    (2, 1),
    (2, 2)
]

B_list = np.arange(0, 500, 10)
tau = 0.99
dim = 40
true_quantile = lambda x: -np.log(1-tau)/ lamb(x)
K = 5

for D_sig, D_gam in depth_pairs:
    print(f"\nRunning crossval for depths (D_sig={D_sig}, D_gam={D_gam})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        K=K
    )

    
    cv_results[(D_sig, D_gam)] = {
        'B_list': B_list,
        'mean_dev': mean_dev,
        'std_dev': std_dev
    }


Running crossval for depths (D_sig=1, D_gam=0)


5it [00:15,  3.16s/it]



Running crossval for depths (D_sig=1, D_gam=1)


5it [00:24,  4.84s/it]



Running crossval for depths (D_sig=2, D_gam=1)


5it [00:25,  5.18s/it]



Running crossval for depths (D_sig=2, D_gam=2)


5it [00:28,  5.72s/it]


In [5]:
fig = go.Figure()
colors = {
    (1, 0): "rgba(194, 127, 64, 0.9)",
    (1, 1): "rgba(70, 130, 180, 0.9)",
    (2, 1): "rgba(54, 144, 133, 0.9)",
    (2, 2): "rgba(123, 102, 162, 0.9)",
}
for (D_sig, D_gam), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=3, color=colors.get((D_sig, D_gam), "black")),
        name=f"({D_sig}, {D_gam})"
    ))
fig.update_layout(
    height=750,
    width=850,
    title=f"CV MISE vs B (K={K} folds)",
    title_x=0.5,
    xaxis_title="B",
    yaxis_title="MISE",
    template="plotly_white",        
    legend=dict(
        title="depth",
        x=1.02,                      
        y=1
    ),
    margin=dict(l=90, r=150, t=80, b=80),
)
fig.show()

In [6]:
cv_results = {}
depth_pairs = [
    (1, 0),
    (1, 1),
    (2, 1),
    (2, 2)
]
B_list = np.arange(0, 500, 10)
tau = 0.99
dim = 40
true_quantile = lambda x: -np.log(1-tau)/ lamb(x)
K = 5
for D_sig, D_gam in depth_pairs:
    print(f"\nRunning crossval for depths (D_sig={D_sig}, D_gam={D_gam})")
    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        l1_reg=30,
        l2_reg=10,
        K=K
    )
    cv_results[(D_sig, D_gam)] = {
        'B_list': B_list,
        'mean_dev': mean_dev,
        'std_dev': std_dev
    }


Running crossval for depths (D_sig=1, D_gam=0)


5it [00:16,  3.31s/it]



Running crossval for depths (D_sig=1, D_gam=1)


5it [00:23,  4.79s/it]



Running crossval for depths (D_sig=2, D_gam=1)


5it [00:25,  5.18s/it]



Running crossval for depths (D_sig=2, D_gam=2)


5it [00:26,  5.35s/it]


In [7]:
fig = go.Figure()
colors = {
    (1, 0): "rgba(194, 127, 64, 0.9)",
    (1, 1): "rgba(70, 130, 180, 0.9)",
    (2, 1): "rgba(54, 144, 133, 0.9)",
    (2, 2): "rgba(123, 102, 162, 0.9)",
}
for (D_sig, D_gam), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=3, color=colors.get((D_sig, D_gam), "black")),
        name=f"({D_sig}, {D_gam})"
    ))
fig.update_layout(
    height=750,
    width=850,
    title_x=0.5,
    xaxis_title="B",
    yaxis_title="MISE",
    template="plotly_white",         
    legend=dict(
        title="depth",
        x=1.02,                     
        y=1
    ),
    margin=dict(l=90, r=150, t=80, b=80),
)
fig.show()

### Experiments on regularization parameters

In [8]:
print("Génération des données")
N = 2000
dim = 40
np.random.seed(42)
X = (np.random.random((N, dim)) - 0.5) * 2
# Fonction paramètre Lambda 
lamb = lambda x: 1 + ((x[0]*x[1] < 0) | (x[2] > 0)).astype(float)
# Y Loi Exponentielle
Y = np.zeros(N)
for i in range(N):
    Y[i] = np.random.exponential(scale=1/lamb(X[i]))

print("Validation Croisée")

# Paramètres 
tau = 0.99
B_list = np.arange(0, 1000, 20)
K = 5

# fct quantile
true_quantile = lambda x: -np.log(1-tau) / lamb(x)

# Configuration des arbres 
fixed_depth = (1, 1) 
D_sig, D_gam = fixed_depth

reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),
    (20, 20),    
    (40, 40)     
]

cv_results = {}

for l1, l2 in reg_pairs:
    print(f"  Running CV pour Regularization (L1={l1}, L2={l2})...")

    
    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, 
        L_gam=10,
        l1_reg=l1, 
        l2_reg=l2,
        K=K
    )

    cv_results[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }

# Plot
print("graph")

fig = go.Figure()

# Ajout des courbes
for (l1, l2), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2.5),
        name=f"L1={l1}, L2={l2}"
    ))

fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)

fig.show()

Génération des données
Validation Croisée
  Running CV pour Regularization (L1=0, L2=0)...


5it [00:44,  8.82s/it]


  Running CV pour Regularization (L1=5, L2=5)...


5it [00:43,  8.66s/it]


  Running CV pour Regularization (L1=10, L2=10)...


...

  Running CV pour Regularization (L1=20, L2=20)...


5it [00:42,  8.44s/it]


  Running CV pour Regularization (L1=40, L2=40)...


5it [00:42,  8.58s/it]

graph


In [9]:
print("Génération des données")
N = 2000
dim = 40
np.random.seed(42)
X = (np.random.random((N, dim)) - 0.5) * 2

#  Lambda 
lamb = lambda x: 1 + ((x[0]*x[1] < 0) | (x[2] > 0)).astype(float)

# Réponse Y (Loi Exponentielle)
Y = np.zeros(N)
for i in range(N):
    Y[i] = np.random.exponential(scale=1/lamb(X[i]))
print("Validation Croisée")

# Paramètres 
tau = 0.99
B_list = np.arange(0, 500, 10)
K = 5

# fct quantile
true_quantile = lambda x: -np.log(1-tau) / lamb(x)
fixed_depth = (1, 0) 
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),     
    (5, 5),
    (10, 10),
    (20, 20),    
    (40, 40)     
]
cv_results = {}

for l1, l2 in reg_pairs:
    print(f" Running CV pour Regularization (L1={l1}, L2={l2})")


    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, 
        L_gam=10,
        l1_reg=l1, 
        l2_reg=l2,
        K=K
    )

    cv_results[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
# Plot
print("3. Génération du graphique...")
fig = go.Figure()
for (l1, l2), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2.5),
        name=f"L1={l1}, L2={l2}"
    ))

fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)
fig.show()

Génération des données
Validation Croisée
 Running CV pour Regularization (L1=0, L2=0)


5it [00:15,  3.11s/it]


 Running CV pour Regularization (L1=5, L2=5)


5it [00:15,  3.05s/it]


 Running CV pour Regularization (L1=10, L2=10)


...

 Running CV pour Regularization (L1=20, L2=20)


5it [00:15,  3.05s/it]


 Running CV pour Regularization (L1=40, L2=40)


5it [00:15,  3.02s/it]

3. Génération du graphique...


In [ ]:
print("Génération des données")
N = 2000
dim = 40
np.random.seed(42)
X = (np.random.random((N, dim)) - 0.5) * 2
#  Lambda
lamb = lambda x: 1 + ((x[0]*x[1] < 0) | (x[2] > 0)).astype(float)
# Y Loi Exponentielle
Y = np.zeros(N)
for i in range(N):
    Y[i] = np.random.exponential(scale=1/lamb(X[i]))

print("Validation Croisée")
# Paramètres 
tau = 0.99
B_list = np.arange(0, 1000, 20)
K = 5
true_quantile = lambda x: -np.log(1-tau) / lamb(x)
fixed_depth = (2, 1) 
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),
    (20, 20),    
    (40, 40)     
]
cv_results = {}

for l1, l2 in reg_pairs:
    print(f"  Running CV pour Regularization (L1={l1}, L2={l2})...")

    
    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, 
        L_gam=10,
        l1_reg=l1, 
        l2_reg=l2,
        K=K
    )

    cv_results[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }

# Plot
print("3. Génération du graphique...")
fig = go.Figure()
for (l1, l2), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2.5),
        name=f"L1={l1}, L2={l2}"
    ))

fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)

fig.show()

Génération des données
Validation Croisée
  Running CV pour Regularization (L1=0, L2=0)...


5it [00:45,  9.06s/it]


  Running CV pour Regularization (L1=5, L2=5)...


5it [00:45,  9.01s/it]


  Running CV pour Regularization (L1=10, L2=10)...


...

  Running CV pour Regularization (L1=20, L2=20)...


5it [00:44,  8.95s/it]


  Running CV pour Regularization (L1=40, L2=40)...


5it [00:45,  9.16s/it]

3. Génération du graphique...


In [11]:
print("Génération des données")
N = 2000
dim = 40
np.random.seed(42)
X = (np.random.random((N, dim)) - 0.5) * 2
#  Lambda
lamb = lambda x: 1 + ((x[0]*x[1] < 0) | (x[2] > 0)).astype(float)
# Y Loi Exponentielle
Y = np.zeros(N)
for i in range(N):
    Y[i] = np.random.exponential(scale=1/lamb(X[i]))
print("Validation Croisée")
# Paramètres 
tau = 0.99
B_list = np.arange(0, 1000, 10)
K = 5
true_quantile = lambda x: -np.log(1-tau) / lamb(x)
fixed_depth = (2, 2) 
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),
    (20, 20),    
    (40, 40)     
]
cv_results = {}
for l1, l2 in reg_pairs:
    print(f"Running CV pour Regularization (L1={l1}, L2={l2})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, 
        L_gam=10,
        l1_reg=l1, 
        l2_reg=l2,
        K=K
    )

    cv_results[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
# Plot
print("Génération du graphique")
fig = go.Figure()
for (l1, l2), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2.5),
        name=f"L1={l1}, L2={l2}"
    ))

fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)

fig.show()

Génération des données
Validation Croisée
Running CV pour Regularization (L1=0, L2=0)


5it [01:16, 15.39s/it]


Running CV pour Regularization (L1=5, L2=5)


5it [01:17, 15.57s/it]


Running CV pour Regularization (L1=10, L2=10)


...

Running CV pour Regularization (L1=20, L2=20)


5it [01:17, 15.49s/it]


Running CV pour Regularization (L1=40, L2=40)


5it [01:16, 15.34s/it]

Génération du graphique


## Model 2 : Student t-distribution with heteroskedastic scaling

## Experiments on trees hyperparameters

In [12]:
tau = 0.99
df = 4
X = (np.random.random((2000, 40)) - 0.5) * 2
scale = lambda x: 1 + (x[0] > 0).astype(float)
Y = np.zeros(2000)
for i in range(2000):
    Y[i] = t.rvs(df=df) * scale(X[i])

In [13]:

cv_results = {}
depth_pairs = [
    (1, 0),
    (1, 1),
]
B_list = np.arange(0, 500, 10)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))
for D_sig, D_gam in depth_pairs:
    print(f"\nRunning crossval for depths (D_sig={D_sig}, D_gam={D_gam})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10,
        L_gam=10,
        l1_reg=0,
        l2_reg=0,
        K=K
    )

    cv_results[(D_sig, D_gam)] = {
        'B_list': B_list,
        'mean_dev': mean_dev,
        'std_dev': std_dev
    }


Running crossval for depths (D_sig=1, D_gam=0)


5it [00:19,  3.82s/it]



Running crossval for depths (D_sig=1, D_gam=1)


5it [00:27,  5.56s/it]


In [14]:
fig = go.Figure()
colors = {
    (1, 0): "rgba(194, 127, 64, 0.9)",
    (1, 1): "rgba(70, 130, 180, 0.9)",
    (2, 1): "rgba(54, 144, 133, 0.9)",
    (2, 2): "rgba(123, 102, 162, 0.9)",
}
for (D_sig, D_gam), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=3, color=colors.get((D_sig, D_gam), "black")),
        name=f"({D_sig}, {D_gam})"
    ))
fig.update_layout(
    height=750,
    width=850,
    title=f"CV MISE vs B (K={K} folds)",
    title_x=0.5,
    xaxis_title="B",
    yaxis_title="MISE",
    template="plotly_white",         
    legend=dict(
        title="depth",
        x=1.02,                      
        y=1
    ),
    margin=dict(l=90, r=150, t=80, b=80),
)

fig.show()

In [15]:
cv_results = {}
depth_pairs = [
    (1, 0),
    (1, 1),
]
B_list = np.arange(0, 500, 10)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))
for D_sig, D_gam in depth_pairs:
    print(f"\nRunning crossval for depths (D_sig={D_sig}, D_gam={D_gam})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig,
        D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10,
        L_gam=10,
        l1_reg=10,
        l2_reg=10,
        K=K
    )
    cv_results[(D_sig, D_gam)] = {
        'B_list': B_list,
        'mean_dev': mean_dev,
        'std_dev': std_dev
    }


Running crossval for depths (D_sig=1, D_gam=0)


5it [00:19,  3.82s/it]



Running crossval for depths (D_sig=1, D_gam=1)


5it [00:27,  5.54s/it]


In [16]:
fig = go.Figure()
colors = {
    (1, 0): "rgba(194, 127, 64, 0.9)",
    (1, 1): "rgba(70, 130, 180, 0.9)",
    (2, 1): "rgba(54, 144, 133, 0.9)",
    (2, 2): "rgba(123, 102, 162, 0.9)",
}
for (D_sig, D_gam), results in cv_results.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=3, color=colors.get((D_sig, D_gam), "black")),
        name=f"({D_sig}, {D_gam})"
    ))
fig.update_layout(
    height=750,
    width=850,
    title=f"CV MISE vs B (K={K} folds)",
    title_x=0.5,
    xaxis_title="B",
    yaxis_title="MISE",
    template="plotly_white",         
    legend=dict(
        title="depth",
        x=1.02,                      
        y=1
    ),
    margin=dict(l=90, r=150, t=80, b=80),
)
fig.show()

### Experiments on regularization parameters

In [17]:
fixed_depth = (1, 1)
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),   
    (20, 20),
    (10,20),
]
cv_results_reg_refined = {}
B_list = np.arange(0, 1000, 10)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))
print(f"Tree{fixed_depth}")

for l1, l2 in reg_pairs:
    print(f" CV for Reg (L1={l1}, L2={l2})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig, D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, L_gam=10,
        l1_reg=l1, l2_reg=l2,
        K=K
    )

    cv_results_reg_refined[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
fig = go.Figure()
for (l1, l2), results in cv_results_reg_refined.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2), 
        name=f"L1={l1}, L2={l2}"
    ))
fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)
fig.show()

Tree(1, 1)
 CV for Reg (L1=0, L2=0)


5it [01:18, 15.74s/it]


 CV for Reg (L1=5, L2=5)


5it [01:18, 15.69s/it]


 CV for Reg (L1=10, L2=10)


5it [01:17, 15.56s/it]


 CV for Reg (L1=20, L2=20)


5it [01:18, 15.65s/it]


 CV for Reg (L1=10, L2=20)


5it [01:18, 15.65s/it]


In [18]:
fixed_depth = (1, 0)
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),    
    (20, 20),
    (10,20),
]
cv_results_reg_refined = {}
B_list = np.arange(0, 1000, 20)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))
print(f"Tree{fixed_depth}")
for l1, l2 in reg_pairs:
    print(f"CV for Reg (L1={l1}, L2={l2})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig, D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, L_gam=10,
        l1_reg=l1, l2_reg=l2,
        K=K
    )

    cv_results_reg_refined[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
fig = go.Figure()
for (l1, l2), results in cv_results_reg_refined.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2), 
        name=f"L1={l1}, L2={l2}"
    ))
fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)

fig.show()

Tree(1, 0)
CV for Reg (L1=0, L2=0)


5it [00:29,  5.81s/it]


CV for Reg (L1=5, L2=5)


5it [00:28,  5.75s/it]


CV for Reg (L1=10, L2=10)


5it [00:28,  5.67s/it]


CV for Reg (L1=20, L2=20)


5it [00:28,  5.75s/it]


CV for Reg (L1=10, L2=20)


5it [00:28,  5.72s/it]


In [19]:
fixed_depth = (2,1)
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0, 0),      
    (5, 5),
    (10, 10),   
    (20, 20),
    (10,20),
]
cv_results_reg_refined = {}
B_list = np.arange(0, 1000, 20)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))
print(f"Tree{fixed_depth}")
for l1, l2 in reg_pairs:
    print(f" CV for Reg (L1={l1}, L2={l2})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig, D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, L_gam=10,
        l1_reg=l1, l2_reg=l2,
        K=K
    )

    cv_results_reg_refined[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
fig = go.Figure()
for (l1, l2), results in cv_results_reg_refined.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2),
        name=f"L1={l1}, L2={l2}"
    ))
fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)
fig.show()

Tree(2, 1)
 CV for Reg (L1=0, L2=0)


5it [00:49,  9.89s/it]


 CV for Reg (L1=5, L2=5)


5it [00:49,  9.97s/it]


 CV for Reg (L1=10, L2=10)


5it [00:50, 10.07s/it]


 CV for Reg (L1=20, L2=20)


5it [00:55, 11.03s/it]


 CV for Reg (L1=10, L2=20)


5it [00:52, 10.44s/it]


In [20]:
fixed_depth = (2,2)
D_sig, D_gam = fixed_depth
reg_pairs = [
    (0,0),
    (30, 30),     
    (5, 5),
    (10, 10),   
    (10,20),
]
cv_results_reg_refined = {}
B_list = np.arange(0, 1000, 20)
dim = 40
K = 5
true_quantile = lambda x: t.ppf(tau, df=df, loc=0, scale=scale(x))

print(f"Tree{fixed_depth}")

for l1, l2 in reg_pairs:
    print(f"CV for Reg (L1={l1}, L2={l2})")

    mean_dev, std_dev = cv_mise_path(
        X, Y, tau,
        D_sig=D_sig, D_gam=D_gam,
        B_list=B_list,
        true_quantile_func=true_quantile,
        dim=dim,
        s=0.6,
        L_sig=10, L_gam=10,
        l1_reg=l1, l2_reg=l2,
        K=K
    )

    cv_results_reg_refined[(l1, l2)] = {
        'B_list': B_list,
        'mean_dev': mean_dev
    }
fig = go.Figure()
for (l1, l2), results in cv_results_reg_refined.items():
    fig.add_trace(go.Scatter(
        x=results['B_list'],
        y=results['mean_dev'],
        mode="lines",
        line=dict(width=2),
        name=f"L1={l1}, L2={l2}"
    ))

fig.update_layout(
    height=750,
    width=900,
    title_x=0.5,
    xaxis_title="B ",
    yaxis_title="MISE",
    template="plotly_white",
    legend=dict(title="Reg parameters", x=1.02, y=1),
    margin=dict(l=90, r=150, t=100, b=80),
)

fig.show()

Tree(2, 2)
CV for Reg (L1=0, L2=0)


5it [00:55, 11.06s/it]


CV for Reg (L1=30, L2=30)


5it [00:54, 10.99s/it]


CV for Reg (L1=5, L2=5)


5it [00:54, 10.84s/it]


CV for Reg (L1=10, L2=10)


5it [00:54, 10.84s/it]


CV for Reg (L1=10, L2=20)


5it [00:52, 10.60s/it]


# BOXPLOT
## Données exponentielles

In [24]:
N_SIM = 200       
N = 2000         
dim = 40        
tau = 0.99       
B_frozen = 200   
L_reg_opt = 10  

gb_configs = [
    
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),

    
    ("GB(1,1) Reg",   (1, 1), 10, 10),
    ("GB(2,1) Reg",   (2, 1), 10, 10),
]
final_results = {config[0]: [] for config in gb_configs}
final_results["QRF"] = [] 

print("Configurations prêtes")

Configurations prêtes


In [25]:
N_SIM = 50
N = 2000
dim = 40
tau = 0.99
B_frozen = 200
gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 10, 10),
    ("GB(2,1) Reg",   (2, 1), 10, 10),
]
final_results = {config[0]: [] for config in gb_configs}
final_results["QRF"] = []
final_results["Constant"] = [] 

print(f"Début des {N_SIM} simulations...")
for m in range(N_SIM):
    if m % 5 == 0: print(f"--- Simulation {m}/{N_SIM} ---")
    np.random.seed(m)
    X = (np.random.random((N, dim)) - 0.5) * 2
    lamb_true = 1 + ((X[:, 0] * X[:, 1] < 0) | (X[:, 2] > 0)).astype(float)
    Y = np.zeros(N)
    for i in range(N):
        Y[i] = np.random.exponential(scale=1/lamb_true[i])
    
    Y_true = -np.log(1-tau) / lamb_true
    qrf_thresh = RandomForestQuantileRegressor(n_estimators=10, min_samples_leaf=20, random_state=m)
    qrf_thresh.fit(X, Y)
    threshold = qrf_thresh.predict(X, quantiles=[0.8])
    Z = Y - threshold
    mask = Z > 0
    X_excess, Z_excess = X[mask], Z[mask]
    term_extrap = (1 - tau) / (1 - 0.8)
    gamma_0, _, sigma_0 = genpareto.fit(Z_excess, floc=0)
    Y_pred_const = threshold + (sigma_0 / gamma_0) * ( (term_extrap ** (-gamma_0)) - 1 )
    
    mise_const = mean_squared_error(Y_true, Y_pred_const)
    final_results["Constant"].append(mise_const)
    for name, depth, l1, l2 in gb_configs:
        D_sig, D_gam = depth
        model = GB_GPD(
            B=B_frozen,
            D_sig=D_sig, D_gam=D_gam,
            lamb_sig=0.01, lamb_gam=0.001,
            s=0.6, L_sig=10, L_gam=10,
            l1_reg=l1, l2_reg=l2
        )
        model.fit(X_excess, Z_excess)
        sigma_pred, gamma_pred = model.predict(X)
        
        Y_pred_gb = threshold + (sigma_pred / gamma_pred) * ( (term_extrap ** (-gamma_pred)) - 1 )
        mise = mean_squared_error(Y_true, Y_pred_gb)
        final_results[name].append(mise)
    qrf = RandomForestQuantileRegressor(n_estimators=200, min_samples_leaf=10, random_state=m)
    qrf.fit(X, Y)
    Y_pred_qrf = qrf.predict(X, quantiles=[tau])
    mise_qrf = mean_squared_error(Y_true, Y_pred_qrf)
    final_results["QRF"].append(mise_qrf)

print("Ready for plotting final results.")

Début des 50 simulations...
--- Simulation 0/50 ---
--- Simulation 5/50 ---
--- Simulation 10/50 ---
--- Simulation 15/50 ---
--- Simulation 20/50 ---
--- Simulation 25/50 ---
--- Simulation 30/50 ---
--- Simulation 35/50 ---
--- Simulation 40/50 ---
--- Simulation 45/50 ---
Ready for plotting final results.


In [30]:
df = pd.DataFrame(final_results)
df_melted = df.melt(var_name="Method", value_name="ISE")
means = df_melted.groupby("Method")["ISE"].mean().reset_index()
LIMIT_X = 10  

fig = px.box(
    df_melted,
    y="Method",
    x="ISE",
    color="Method",
    orientation='h',
    template="plotly_white",
    points="all",
    range_x=[-0.5, LIMIT_X]
)
for i, row in means.iterrows():
    if row["ISE"] < LIMIT_X:
        fig.add_shape(
            type="line",
            y0=-0.4 + i, y1=0.4 + i,
            x0=row["ISE"], x1=row["ISE"],
            line=dict(color="black", width=3),
            xref="x", yref="y"
        )

fig.update_layout(
    xaxis_title="ISE (Integrated Squared Error)",
    yaxis_title="Method",
    showlegend=False,
    height=500,
    width=900
)
fig.update_yaxes(autorange="reversed")

fig.show()

### Quantile 0.995

In [31]:
N_SIM = 200       
N = 2000         
dim = 40        
tau = 0.99       
B_frozen = 200   
L_reg_opt = 10  

gb_configs = [
    
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),

    
    ("GB(1,1) Reg",   (1, 1), 10, 10),
    ("GB(2,1) Reg",   (2, 1), 10, 10),
]
final_results = {config[0]: [] for config in gb_configs}
final_results["QRF"] = [] 

print("Configurations prêtes")

Configurations prêtes


In [32]:
N_SIM = 50
N = 2000
dim = 40
tau = 0.995
B_frozen = 200
gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 10, 10),
    ("GB(2,1) Reg",   (2, 1), 10, 10),
]
final_results = {config[0]: [] for config in gb_configs}
final_results["QRF"] = []
final_results["Constant"] = [] 

print(f"Début des {N_SIM} simulations...")
for m in range(N_SIM):
    if m % 5 == 0: print(f"--- Simulation {m}/{N_SIM} ---")
    np.random.seed(m)
    X = (np.random.random((N, dim)) - 0.5) * 2
    lamb_true = 1 + ((X[:, 0] * X[:, 1] < 0) | (X[:, 2] > 0)).astype(float)
    Y = np.zeros(N)
    for i in range(N):
        Y[i] = np.random.exponential(scale=1/lamb_true[i])
    
    Y_true = -np.log(1-tau) / lamb_true
    qrf_thresh = RandomForestQuantileRegressor(n_estimators=10, min_samples_leaf=20, random_state=m)
    qrf_thresh.fit(X, Y)
    threshold = qrf_thresh.predict(X, quantiles=[0.8])
    Z = Y - threshold
    mask = Z > 0
    X_excess, Z_excess = X[mask], Z[mask]
    term_extrap = (1 - tau) / (1 - 0.8)
    gamma_0, _, sigma_0 = genpareto.fit(Z_excess, floc=0)
    Y_pred_const = threshold + (sigma_0 / gamma_0) * ( (term_extrap ** (-gamma_0)) - 1 )
    
    mise_const = mean_squared_error(Y_true, Y_pred_const)
    final_results["Constant"].append(mise_const)
    for name, depth, l1, l2 in gb_configs:
        D_sig, D_gam = depth
        model = GB_GPD(
            B=B_frozen,
            D_sig=D_sig, D_gam=D_gam,
            lamb_sig=0.01, lamb_gam=0.001,
            s=0.6, L_sig=10, L_gam=10,
            l1_reg=l1, l2_reg=l2
        )
        model.fit(X_excess, Z_excess)
        sigma_pred, gamma_pred = model.predict(X)
        
        Y_pred_gb = threshold + (sigma_pred / gamma_pred) * ( (term_extrap ** (-gamma_pred)) - 1 )
        mise = mean_squared_error(Y_true, Y_pred_gb)
        final_results[name].append(mise)
    qrf = RandomForestQuantileRegressor(n_estimators=200, min_samples_leaf=10, random_state=m)
    qrf.fit(X, Y)
    Y_pred_qrf = qrf.predict(X, quantiles=[tau])
    mise_qrf = mean_squared_error(Y_true, Y_pred_qrf)
    final_results["QRF"].append(mise_qrf)

print("Ready for plotting final results.")

Début des 50 simulations...
--- Simulation 0/50 ---
--- Simulation 5/50 ---
--- Simulation 10/50 ---
--- Simulation 15/50 ---
--- Simulation 20/50 ---
--- Simulation 25/50 ---
--- Simulation 30/50 ---
--- Simulation 35/50 ---
--- Simulation 40/50 ---
--- Simulation 45/50 ---
Ready for plotting final results.


In [33]:
df = pd.DataFrame(final_results)
df_melted = df.melt(var_name="Method", value_name="ISE")
means = df_melted.groupby("Method")["ISE"].mean().reset_index()
LIMIT_X = 10  

fig = px.box(
    df_melted,
    y="Method",
    x="ISE",
    color="Method",
    orientation='h',
    template="plotly_white",
    points="all",
    range_x=[-0.5, LIMIT_X]
)
for i, row in means.iterrows():
    if row["ISE"] < LIMIT_X:
        fig.add_shape(
            type="line",
            y0=-0.4 + i, y1=0.4 + i,
            x0=row["ISE"], x1=row["ISE"],
            line=dict(color="black", width=3),
            xref="x", yref="y"
        )

fig.update_layout(
    xaxis_title="ISE (Integrated Squared Error)",
    yaxis_title="Method",
    showlegend=False,
    height=500,
    width=900
)
fig.update_yaxes(autorange="reversed")

fig.show()

## Données student

### Quantile 0.99

In [34]:
N_SIM = 100      
N = 2000
dim = 40
tau = 0.99
df = 4          
B_frozen = 200
L_reg_opt = 10

gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 5,5),
    ("GB(2,1) Reg",   (2, 1), 10,10),
]

final_results_student = {config[0]: [] for config in gb_configs}
final_results_student["QRF"] = []

In [28]:
N_SIM = 50
N = 2000
dim = 40
tau = 0.99
df = 4 
B_frozen = 200
gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 10, 20),
    ("GB(2,1) Reg",   (2, 1), 10, 20),
]
final_results_student = {config[0]: [] for config in gb_configs}
final_results_student["QRF"] = []
final_results_student["Constant"] = [] 
print(f"Début des {N_SIM} simulations (Student-t)...")
for m in range(N_SIM):
    if m % 5 == 0: print(f"--- Simulation {m}/{N_SIM} ---")
    np.random.seed(m)
    X = (np.random.random((N, dim)) - 0.5) * 2
    scale_true = 1 + (X[:, 0] > 0).astype(float)
    
    noise = t.rvs(df=df, size=N)
    Y = noise * scale_true
    quantile_student_std = t.ppf(tau, df=df) 
    Y_true = quantile_student_std * scale_true
    qrf_thresh = RandomForestQuantileRegressor(n_estimators=10, min_samples_leaf=20, random_state=m)
    qrf_thresh.fit(X, Y)
    threshold = qrf_thresh.predict(X, quantiles=[0.8])
    Z = Y - threshold
    mask = Z > 0
    X_excess, Z_excess = X[mask], Z[mask]
    term_extrap = (1 - tau) / (1 - 0.8)
    gamma_0, _, sigma_0 = genpareto.fit(Z_excess, floc=0)
    Y_pred_const = threshold + (sigma_0 / gamma_0) * ( (term_extrap ** (-gamma_0)) - 1 )
    
    mise_const = mean_squared_error(Y_true, Y_pred_const)
    final_results_student["Constant"].append(mise_const)

    for name, depth, l1, l2 in gb_configs:
        D_sig, D_gam = depth
        model = GB_GPD(
            B=B_frozen,
            D_sig=D_sig, D_gam=D_gam,
            lamb_sig=0.01, lamb_gam=0.001,
            s=0.6, L_sig=10, L_gam=10,
            l1_reg=l1, l2_reg=l2
        )
        model.fit(X_excess, Z_excess)
        sigma_pred, gamma_pred = model.predict(X)
        
        Y_pred_gb = threshold + (sigma_pred / gamma_pred) * ( (term_extrap ** (-gamma_pred)) - 1 )
        
        mise = mean_squared_error(Y_true, Y_pred_gb)
        final_results_student[name].append(mise)

    qrf = RandomForestQuantileRegressor(n_estimators=200, min_samples_leaf=10, random_state=m)
    qrf.fit(X, Y)
    Y_pred_qrf = qrf.predict(X, quantiles=[tau])
    mise_qrf = mean_squared_error(Y_true, Y_pred_qrf)
    final_results_student["QRF"].append(mise_qrf)

print("Ready for plotting final results (Student).")

Début des 50 simulations (Student-t)...
--- Simulation 0/50 ---
--- Simulation 5/50 ---
--- Simulation 10/50 ---
--- Simulation 15/50 ---
--- Simulation 20/50 ---
--- Simulation 25/50 ---
--- Simulation 30/50 ---
--- Simulation 35/50 ---
--- Simulation 40/50 ---
--- Simulation 45/50 ---
Ready for plotting final results (Student).


In [29]:
df_student = pd.DataFrame(final_results_student)
df_melted_st = df_student.melt(var_name="Method", value_name="ISE")
means_st = df_melted_st.groupby("Method")["ISE"].mean().reset_index()
LIMIT_X_ST = 20  

fig = px.box(
    df_melted_st,
    y="Method",
    x="ISE",
    color="Method",
    orientation='h',
    template="plotly_white",
    points="all",
    range_x=[-0.5, LIMIT_X_ST]
)
for i, row in means_st.iterrows():
    if row["ISE"] < LIMIT_X_ST:
        fig.add_shape(
            type="line",
            y0=-0.4 + i, y1=0.4 + i,
            x0=row["ISE"], x1=row["ISE"],
            line=dict(color="black", width=3),
            xref="x", yref="y"
        )

fig.update_layout(
    xaxis_title="ISE (Integrated Squared Error)",
    yaxis_title="Method",
    showlegend=False,
    height=500,
    width=900
)
fig.update_yaxes(autorange="reversed")
fig.show()

### Quantile 0.995

In [35]:
N_SIM = 100      
N = 2000
dim = 40
tau = 0.995
df = 4          
B_frozen = 200
L_reg_opt = 10

gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 5,5),
    ("GB(2,1) Reg",   (2, 1), 10,10),
]

final_results_student = {config[0]: [] for config in gb_configs}
final_results_student["QRF"] = []

In [36]:
N_SIM = 50
N = 2000
dim = 40
tau = 0.995
df = 4 
B_frozen = 200
gb_configs = [
    ("GB(1,1) NoReg", (1, 1), 0, 0),
    ("GB(2,1) NoReg", (2, 1), 0, 0),
    ("GB(1,1) Reg",   (1, 1), 10, 20),
    ("GB(2,1) Reg",   (2, 1), 10, 20),
]
final_results_student = {config[0]: [] for config in gb_configs}
final_results_student["QRF"] = []
final_results_student["Constant"] = [] 
print(f"Début des {N_SIM} simulations (Student-t)...")
for m in range(N_SIM):
    if m % 5 == 0: print(f"--- Simulation {m}/{N_SIM} ---")
    np.random.seed(m)
    X = (np.random.random((N, dim)) - 0.5) * 2
    scale_true = 1 + (X[:, 0] > 0).astype(float)
    
    noise = t.rvs(df=df, size=N)
    Y = noise * scale_true
    quantile_student_std = t.ppf(tau, df=df) 
    Y_true = quantile_student_std * scale_true
    qrf_thresh = RandomForestQuantileRegressor(n_estimators=10, min_samples_leaf=20, random_state=m)
    qrf_thresh.fit(X, Y)
    threshold = qrf_thresh.predict(X, quantiles=[0.8])
    Z = Y - threshold
    mask = Z > 0
    X_excess, Z_excess = X[mask], Z[mask]
    term_extrap = (1 - tau) / (1 - 0.8)
    gamma_0, _, sigma_0 = genpareto.fit(Z_excess, floc=0)
    Y_pred_const = threshold + (sigma_0 / gamma_0) * ( (term_extrap ** (-gamma_0)) - 1 )
    
    mise_const = mean_squared_error(Y_true, Y_pred_const)
    final_results_student["Constant"].append(mise_const)

    for name, depth, l1, l2 in gb_configs:
        D_sig, D_gam = depth
        model = GB_GPD(
            B=B_frozen,
            D_sig=D_sig, D_gam=D_gam,
            lamb_sig=0.01, lamb_gam=0.001,
            s=0.6, L_sig=10, L_gam=10,
            l1_reg=l1, l2_reg=l2
        )
        model.fit(X_excess, Z_excess)
        sigma_pred, gamma_pred = model.predict(X)
        
        Y_pred_gb = threshold + (sigma_pred / gamma_pred) * ( (term_extrap ** (-gamma_pred)) - 1 )
        
        mise = mean_squared_error(Y_true, Y_pred_gb)
        final_results_student[name].append(mise)

    qrf = RandomForestQuantileRegressor(n_estimators=200, min_samples_leaf=10, random_state=m)
    qrf.fit(X, Y)
    Y_pred_qrf = qrf.predict(X, quantiles=[tau])
    mise_qrf = mean_squared_error(Y_true, Y_pred_qrf)
    final_results_student["QRF"].append(mise_qrf)

print("Ready for plotting final results (Student).")

Début des 50 simulations (Student-t)...
--- Simulation 0/50 ---
--- Simulation 5/50 ---
--- Simulation 10/50 ---
--- Simulation 15/50 ---
--- Simulation 20/50 ---
--- Simulation 25/50 ---
--- Simulation 30/50 ---
--- Simulation 35/50 ---
--- Simulation 40/50 ---
--- Simulation 45/50 ---
Ready for plotting final results (Student).


In [37]:
df_student = pd.DataFrame(final_results_student)
df_melted_st = df_student.melt(var_name="Method", value_name="ISE")
means_st = df_melted_st.groupby("Method")["ISE"].mean().reset_index()
LIMIT_X_ST = 20  

fig = px.box(
    df_melted_st,
    y="Method",
    x="ISE",
    color="Method",
    orientation='h',
    template="plotly_white",
    points="all",
    range_x=[-0.5, LIMIT_X_ST]
)
for i, row in means_st.iterrows():
    if row["ISE"] < LIMIT_X_ST:
        fig.add_shape(
            type="line",
            y0=-0.4 + i, y1=0.4 + i,
            x0=row["ISE"], x1=row["ISE"],
            line=dict(color="black", width=3),
            xref="x", yref="y"
        )

fig.update_layout(
    xaxis_title="ISE (Integrated Squared Error)",
    yaxis_title="Method",
    showlegend=False,
    height=500,
    width=900
)
fig.update_yaxes(autorange="reversed")
fig.show()